# **Études de cooccurrences de mots**
Auteur : PM Nugues

Une petite analyse des cooccurrences des racines *priv* et *publi[ck]* dans les débats du riksdag au XVI<sup>e</sup>, XVII<sup>e</sup> et XVIII<sup>e</sup> siècles. On peut appliquer ce programme à n'importe quelle séquence de caractères. Il suffit de changer quelques paramètres.

# Préliminaires

Les modules nécessaires

In [1]:
import operator
import regex as re
import sys
from math import log, sqrt
from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir

# Lecture des fichers XML et extraction des textes

## Liste des fichiers

On extrait la liste des fichers de débats. Les fichiers sont rassemblés pas décénnie.

In [2]:
dossiers = ['10_1-6_3 1970 andra kammaren', '10_1-12_3 1970 första kammaren']
dossiers = ['Riksdagstryck/' + dossier + '/' for dossier in dossiers]
dossiers

['Riksdagstryck/10_1-6_3 1970 andra kammaren/',
 'Riksdagstryck/10_1-12_3 1970 första kammaren/']

In [3]:
fichiers_par_décénnie = [listdir(dossier) for dossier in dossiers]
fichiers_par_décénnie2 = []
fichiers_tous = []
for i in range(len(dossiers)):
    fichiers_par_décénnie2 += [[dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]]
    fichiers_tous += [dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]
fichiers_par_décénnie2

[['Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__9.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__8.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__3.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__2.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__1.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__5.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__4.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__6.xml',
  'Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__7.xml'],
 ['Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__10.xml',
  'Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__1.xml',
  'Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__2.xml',
  'Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__3.xml',
  'Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__7.xml',
  'Riksdagstryck/

## Lecture des fichiers

On fait l'analyse syntaxique du code XML de chaque fichier

In [4]:
soups = []
for fichiers in fichiers_par_décénnie2:
    mini_soup = []
    for fichier in fichiers:
        print(fichier)
        with open(fichier, 'r', encoding='utf8') as infile:
            corpus_xml = infile.read()[1:]
        soup = BeautifulSoup(corpus_xml, 'xml')
        mini_soup += soup
    soups += [mini_soup]

Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__9.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__8.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__3.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__2.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__1.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__5.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__4.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__6.xml
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__7.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__10.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__1.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__2.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__3.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__7.xml
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__6.xml
Riksdagstryck/10_1-12

On vérifie le nombre de décénnies

In [5]:
len(soups)

2

On concatène les lignes de chaque fichier de sorte qu'il n'y ait qu'une ligne de texte par fichier

In [6]:
textes = []
cnt = 0
print('Nom du fichier et nombre de lignes')
for mini_soups in soups:
    textes_par_décénnie = []
    for soup in mini_soups:
        lines = soup.find_all('line')
        print(fichiers_tous[cnt], ':', len(lines))
        cnt += 1
        texte = ''
        for line in lines:
            texte = texte + line.get_text() + '\n'
        #textes_par_décénnie += texte
        textes_par_décénnie += [texte]
    textes += [textes_par_décénnie]

Nom du fichier et nombre de lignes
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__9.xml : 12423
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__8.xml : 9032
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__3.xml : 16736
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__2.xml : 14527
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__1.xml : 3087
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__5.xml : 3908
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__4.xml : 4344
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__6.xml : 1590
Riksdagstryck/10_1-6_3 1970 andra kammaren/prot_1970__ak__7.xml : 9908
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__10.xml : 11865
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__1.xml : 1914
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__2.xml : 14319
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__3.xml : 14714
Riksdagstryck/10_1-12_3

On vérifie le nombre de textes

In [7]:
len(textes)

2

Nombre de textes par décénnie

In [8]:
list(map(len, textes))

[9, 10]

# Concordances

Les séquences de caractères à rechercher et le contexte de la concordance

On fixe le choix à *priv* et *publi(c|k|que)*. On peut les remplacer par n'importe quelle séquence

In [9]:
PRIVAT = True

In [10]:
if PRIVAT:
    chaîne = 'privat'
else:
    chaîne = 'publi(k|c|que)'
contexte = 25

In [11]:
chaîne = re.sub(' ', '\\s+', chaîne)

In [12]:
for i, décénnie in enumerate(textes):
    print('===DÉCÉNNIE===', i + 1)
    for i, texte in enumerate(décénnie):
        print(fichiers[i], ':')
        texte = re.sub('\s+', ' ', texte.lower())
        concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
                       .format(pattern=chaîne, width=contexte))
        for match in re.finditer(concordance, texte):
            print('  ', match.group(1))

===DÉCÉNNIE=== 1
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__10.xml :
   eliga, ko¬ operativa och privata företagsformer tävlar p
    både vad det gäller den privata och den kooperativa för
   s. de har blivit vanliga privatkapitalister och tän¬ ker
   ¬ göra lösningen när den privata före¬ tagsamheten missl
   som ökat utrymme ges för privat initiativ och framåtanda
   tidens reglering. på den privata ar¬ betsmarknaden är de
   rågor finns redan på det privata området. den av oss mot
   lst önskan inom vare sig privat eller koope¬ rativ hande
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__1.xml :
    ren konfiskation av det privata kapitalet i jordbruksnä
Riksdagstryck/10_1-12_3 1970 första kammaren/prot_1970__fk__2.xml :
   t en »stark och livaktig privat sektor är ett remissdeba
   ttats för att underlätta privat företagsamhet». jag före
   ar varken statliga eller privata lokalise¬ ringar i någo
   täc¬ ker kostnaderna för privatfinansiering¬ en, vil

# Cooccurrences

## Mots vides

On définit une liste de mots vides. On ne l'utilise pas dans le reste du programme

In [13]:
mots_vides = ['och', 'den', 'också', 'för']

## Fonctions de comptage des mots et des bigrammes

On définit les fonctions de comptage. Elles sont tirées de l'édition à venir de mon livre.

In [14]:
def découpe_mots(texte):
    mots = re.findall("\p{L}+|\p{N}+", texte)
    return mots


"""def découpe_mots(texte):
    mots = re.split('[\s\-,;:!?.’\'«»()–...&‘’“”*—]+', texte)
    mots.remove('')
    return mots"""


def compte_mots(mots):
    c_mots = {}
    for mot in mots:
        c_mots[mot] = c_mots.get(mot, 0) + 1
    return c_mots


def compte_bigrammes(mots):
    c_bigrammes = {}
    for i in range(len(mots) - 1):
        bigramme = (mots[i], mots[i + 1])
        c_bigrammes[bigramme] = c_bigrammes.get(bigramme, 0) + 1
    return c_bigrammes


def compte_bigrammes_fen(mots, fen=5):
    c_bigrammes_fen = {}
    for inx in range(1, fen + 1):
        for i in range(len(mots) - inx):
            bigramme = (mots[i], mots[i + inx])
            c_bigrammes_fen[bigramme] = c_bigrammes_fen.get(bigramme, 0) + 1
    return c_bigrammes_fen

On concatène les textes par décénnie. Pour chaque décénnie, on aura donc une ligne de texte

In [15]:
textes = [' '.join(décénnie) for décénnie in textes]

On crée aussi une chaîne avec le corpus en entier

In [16]:
corpus_entier = ' '.join(textes)

## Les mots

On compte les mots par décénnie

In [17]:
c_mots_ordonnés_par_corpus = []
c_mots_par_corpus = []
textes_découpés = []
total_mots_par_corpus = []
for i, texte in enumerate(textes):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    texte = texte.lower()
    texte_découpé = découpe_mots(texte)
    total_mots_par_corpus += [len(texte_découpé)]
    textes_découpés += [texte_découpé]
    # print(mots)
    c_mots = compte_mots(texte_découpé)
    c_mots_par_corpus += [c_mots]
    c_mots_ordonnés = sorted(c_mots.items(), key=operator.itemgetter(1), reverse=True)
    c_mots_ordonnés_par_corpus += [c_mots_ordonnés]
    print('  ', c_mots_ordonnés[:10])
print('Nombre total de mots:', total_mots_par_corpus)

===DÉCÉNNIE=== 1
   [('att', 13685), ('i', 12060), ('och', 9055), ('det', 8639), ('av', 8286), ('som', 7304), ('för', 7134), ('den', 6123), ('en', 6108), ('till', 5894)]
===DÉCÉNNIE=== 2
   [('att', 13102), ('i', 10693), ('och', 8580), ('det', 8503), ('av', 7719), ('som', 6853), ('för', 6434), ('den', 6015), ('en', 5982), ('till', 5504)]
Nombre total de mots: [426717, 401201]


Pour tout le corpus

In [18]:
corpus_entier = corpus_entier.lower()
corpus_découpé = découpe_mots(corpus_entier)
corpus_mots = compte_mots(corpus_découpé)
corpus_mots_ordonnés = sorted(corpus_mots.items(), key=operator.itemgetter(1), reverse=True)
print('  ', corpus_mots_ordonnés[:10])

   [('att', 26787), ('i', 22753), ('och', 17635), ('det', 17142), ('av', 16005), ('som', 14157), ('för', 13568), ('den', 12138), ('en', 12090), ('till', 11398)]


On filtre les mots qui contiennent la séquence de caractères par décénnie

In [19]:
occurrences_chaîne = []
for i, c_mots_ordonnés in enumerate(c_mots_ordonnés_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    #print(fichiers[i], ':')
    occurrences_chaîne_1 = list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés))
    occurrences_chaîne += [occurrences_chaîne_1]
    print(' ', list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés)))

===DÉCÉNNIE=== 1
  [('privata', 41), ('privat', 14), ('privatskolorna', 7), ('privatskolor', 4), ('privatsko', 2), ('privatskoledöd', 2), ('privatkapitalister', 1), ('privatfinansiering', 1), ('privatpersoner', 1), ('privatkapitalistiska', 1), ('privatskoledödens', 1), ('privatskole', 1), ('privatpraktiserande', 1), ('privatsak', 1)]
===DÉCÉNNIE=== 2
  [('privata', 15), ('privat', 8), ('privatpraktiserande', 6), ('privatperson', 3), ('privatpraktikerna', 2), ('privatkapitalistiska', 2), ('privatbilism', 1), ('privatpersoner', 1), ('privatskolorna', 1), ('privatfinansiering', 1), ('privatpraktise', 1), ('privatper', 1)]


Pour tout le corpus

In [20]:
list(filter(lambda x: re.search(chaîne, x[0]), corpus_mots_ordonnés))

[('privata', 56),
 ('privat', 22),
 ('privatskolorna', 8),
 ('privatpraktiserande', 7),
 ('privatskolor', 4),
 ('privatkapitalistiska', 3),
 ('privatperson', 3),
 ('privatfinansiering', 2),
 ('privatpersoner', 2),
 ('privatsko', 2),
 ('privatskoledöd', 2),
 ('privatpraktikerna', 2),
 ('privatkapitalister', 1),
 ('privatskoledödens', 1),
 ('privatskole', 1),
 ('privatsak', 1),
 ('privatbilism', 1),
 ('privatpraktise', 1),
 ('privatper', 1)]

### Fréquence absolue de la séquence

In [21]:
occurrences_chaîne

[[('privata', 41),
  ('privat', 14),
  ('privatskolorna', 7),
  ('privatskolor', 4),
  ('privatsko', 2),
  ('privatskoledöd', 2),
  ('privatkapitalister', 1),
  ('privatfinansiering', 1),
  ('privatpersoner', 1),
  ('privatkapitalistiska', 1),
  ('privatskoledödens', 1),
  ('privatskole', 1),
  ('privatpraktiserande', 1),
  ('privatsak', 1)],
 [('privata', 15),
  ('privat', 8),
  ('privatpraktiserande', 6),
  ('privatperson', 3),
  ('privatpraktikerna', 2),
  ('privatkapitalistiska', 2),
  ('privatbilism', 1),
  ('privatpersoner', 1),
  ('privatskolorna', 1),
  ('privatfinansiering', 1),
  ('privatpraktise', 1),
  ('privatper', 1)]]

### Fréquence relative de la séquence par décénnie

In [22]:
for i in range(len(total_mots_par_corpus)):
    print(sum([x[1] for x in occurrences_chaîne[i]])/total_mots_par_corpus[i])

0.0001827909363817237
0.00010468568124207068


## Les bigrammes

### Les bigrammes comme mots adjacents

On compte les bigrammes et on les ordonne par fréquence pour chaque décénnie

In [23]:
c_bigrammes_ordonnés_par_corpus = []
c_bigrammes_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    print('===DÉCÉNNIE===', i + 1)
    # print(fichiers[i], ':')
    c_bigrammes = compte_bigrammes(texte_découpé)
    c_bigrammes_par_corpus += [c_bigrammes]
    c_bigrammes_ordonnés = sorted(c_bigrammes.items(), 
                                  key=operator.itemgetter(1), reverse=True)
    c_bigrammes_ordonnés_par_corpus += [c_bigrammes_ordonnés]
    print('  ', c_bigrammes_ordonnés[:15])

===DÉCÉNNIE=== 1
   [(('det', 'är'), 1453), (('att', 'det'), 1150), (('för', 'att'), 1117), (('av', 'herr'), 1028), (('herr', 'talman'), 956), (('m', 'fl'), 677), (('att', 'man'), 634), (('att', 'vi'), 511), (('onsdagen', 'den'), 455), (('fl', 'om'), 446), (('är', 'det'), 433), (('att', 'de'), 433), (('att', 'få'), 419), (('januari', '1970'), 417), (('att', 'den'), 414)]
===DÉCÉNNIE=== 2
   [(('det', 'är'), 1368), (('av', 'herr'), 1174), (('att', 'det'), 1132), (('för', 'att'), 1014), (('herr', 'talman'), 878), (('att', 'man'), 596), (('m', 'm'), 553), (('m', 'fl'), 506), (('onsdagen', 'den'), 484), (('kungl', 'maj'), 469), (('att', 'vi'), 459), (('är', 'det'), 449), (('i', 'den'), 437), (('att', 'de'), 417), (('att', 'den'), 396)]


Et pour le corpus en entier

In [24]:
corpus_bigrammes = compte_bigrammes(corpus_découpé)
corpus_bigrammes_ordonnés = sorted(corpus_bigrammes.items(), 
                              key=operator.itemgetter(1), reverse=True)
corpus_bigrammes_ordonnés[:15]

[(('det', 'är'), 2821),
 (('att', 'det'), 2282),
 (('av', 'herr'), 2202),
 (('för', 'att'), 2131),
 (('herr', 'talman'), 1834),
 (('att', 'man'), 1230),
 (('m', 'fl'), 1183),
 (('att', 'vi'), 970),
 (('onsdagen', 'den'), 939),
 (('är', 'det'), 882),
 (('att', 'de'), 850),
 (('m', 'm'), 836),
 (('kungl', 'maj'), 832),
 (('att', 'den'), 810),
 (('att', 'få'), 793)]

Les bigrammes qui contienne la séquence de caractères pour chaque décénnie

In [25]:
for i, c_bigrammes_ordonnés in enumerate(c_bigrammes_ordonnés_par_corpus):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(' ', list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1]), 
                      c_bigrammes_ordonnés)))

===DÉCÉNNIE=== 1
  [(('den', 'privata'), 11), (('det', 'privata'), 7), (('privata', 'eller'), 6), (('privata', 'och'), 4), (('de', 'privata'), 4), (('med', 'privata'), 3), (('och', 'privata'), 2), (('privat', 'eller'), 2), (('privat', 'företagsamhet'), 2), (('privata', 'näringslivet'), 2), (('privata', 'sparandet'), 2), (('privata', 'företag'), 2), (('ett', 'privat'), 2), (('privatskolor', 'i'), 2), (('till', 'privatskolorna'), 2), (('mot', 'privatskolorna'), 2), (('för', 'privata'), 2), (('privata', 'stiftelser'), 2), (('intressen', 'privata'), 2), (('privata', 'företagsformer'), 1), (('vanliga', 'privatkapitalister'), 1), (('privatkapitalister', 'och'), 1), (('privata', 'före'), 1), (('för', 'privat'), 1), (('privat', 'initiativ'), 1), (('privata', 'ar'), 1), (('privata', 'området'), 1), (('sig', 'privat'), 1), (('privata', 'kapitalet'), 1), (('livaktig', 'privat'), 1), (('privat', 'sektor'), 1), (('underlätta', 'privat'), 1), (('eller', 'privata'), 1), (('privata', 'lokalise'), 1), 

Et pour le corpus en entier. On ne retient que les couples avec une fréquence supérieure à 5

In [26]:
list(filter(lambda x: (re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1])) and x[1] >= 5, 
                      corpus_bigrammes_ordonnés))

[(('den', 'privata'), 16),
 (('det', 'privata'), 10),
 (('privata', 'eller'), 6),
 (('de', 'privata'), 5)]

### Les bigrammes définis dans une fenêtre

Les bigrammes avec une fenêtre. On considère maintenant le contexte à droite et à gauche du mot avec une fenêtre de 5 mots

Les bigrammes les plus fréquents pour chaque décénnie

In [27]:
c_bigrammes_fen_ordonnés_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    c_bigrammes_fen = compte_bigrammes_fen(texte_découpé)
    c_bigrammes_fen_ordonnés = sorted(c_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
    c_bigrammes_fen_ordonnés_par_corpus += [c_bigrammes_fen_ordonnés]
    print(c_bigrammes_fen_ordonnés[:10])

===DÉCÉNNIE=== 1
[(('det', 'att'), 2184), (('att', 'det'), 1939), (('det', 'är'), 1921), (('jag', 'att'), 1856), (('för', 'att'), 1797), (('att', 'i'), 1764), (('är', 'att'), 1664), (('nr', 'av'), 1481), (('att', 'en'), 1367), (('som', 'i'), 1345)]
===DÉCÉNNIE=== 2
[(('det', 'att'), 2131), (('att', 'det'), 1909), (('det', 'är'), 1824), (('för', 'att'), 1674), (('att', 'i'), 1670), (('jag', 'att'), 1601), (('är', 'att'), 1552), (('av', 'herr'), 1533), (('nr', 'av'), 1375), (('att', 'en'), 1374)]


Et pour le corpus entier

In [28]:
corpus_bigrammes_fen = compte_bigrammes_fen(corpus_découpé)
corpus_bigrammes_ordonnés_fen = sorted(corpus_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
corpus_bigrammes_ordonnés_fen[:15]

[(('det', 'att'), 4315),
 (('att', 'det'), 3848),
 (('det', 'är'), 3745),
 (('för', 'att'), 3471),
 (('jag', 'att'), 3457),
 (('att', 'i'), 3434),
 (('är', 'att'), 3216),
 (('nr', 'av'), 2856),
 (('av', 'herr'), 2776),
 (('att', 'en'), 2741),
 (('som', 'i'), 2620),
 (('att', 'att'), 2595),
 (('herr', 'talman'), 2516),
 (('i', 'det'), 2399),
 (('i', 'och'), 2391)]

Les bigrammes avec la séquence de caractères recherchée par décénnie

In [29]:
for i, c_bigrammes_fen_ordonnés in enumerate(c_bigrammes_fen_ordonnés_par_corpus):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                      re.search(chaîne, x[0][1]), 
                      c_bigrammes_fen_ordonnés)))

===DÉCÉNNIE=== 1
[(('den', 'privata'), 12), (('det', 'privata'), 10), (('med', 'privata'), 8), (('privata', 'och'), 6), (('privata', 'eller'), 6), (('privata', 'i'), 6), (('och', 'privata'), 5), (('på', 'privata'), 5), (('för', 'privata'), 5), (('privata', 'till'), 5), (('de', 'privata'), 4), (('av', 'privata'), 4), (('att', 'privata'), 4), (('privata', 'skall'), 4), (('ekonomiska', 'privata'), 4), (('privat', 'att'), 4), (('som', 'privata'), 4), (('privata', 'med'), 4), (('privata', 'sparandet'), 3), (('till', 'privatskolorna'), 3), (('privatskolorna', 'är'), 3), (('privata', 'som'), 3), (('och', 'privat'), 3), (('intressen', 'privata'), 3), (('privata', 'intres'), 3), (('privata', 'den'), 3), (('i', 'privata'), 3), (('privata', 'statliga'), 3), (('privatskolorna', 'ett'), 3), (('privata', 'är'), 3), (('privata', 'det'), 3), (('privata', 'jag'), 3), (('som', 'privat'), 3), (('privata', 'investeringar'), 3), (('person', 'privata'), 3), (('för', 'privat'), 2), (('privata', 'ar'), 2), ((

Et pour le corpus en entier

In [30]:
list(filter(lambda x: (re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1])) and x[1] >= 5, 
                      corpus_bigrammes_ordonnés_fen))

[(('den', 'privata'), 17),
 (('det', 'privata'), 14),
 (('med', 'privata'), 9),
 (('för', 'privata'), 8),
 (('privata', 'i'), 8),
 (('privata', 'eller'), 7),
 (('på', 'privata'), 7),
 (('som', 'privata'), 7),
 (('och', 'privata'), 6),
 (('privata', 'och'), 6),
 (('de', 'privata'), 6),
 (('privat', 'och'), 6),
 (('av', 'privata'), 6),
 (('privata', 'det'), 6),
 (('ett', 'privat'), 5),
 (('de', 'privatpraktiserande'), 5),
 (('privata', 'den'), 5),
 (('att', 'privat'), 5),
 (('privata', 'med'), 5),
 (('privata', 'har'), 5),
 (('privata', 'till'), 5)]

# Mesures de cooccurrence

On applique maintenant trois mesures de cooccurrence :
* l'information mutuelle de Fano
* les t-scores
* le rapport de vraisemblance

## Information mutuelle

Définition de l'information mutuelle

In [31]:
def info_mutuelle(c_mots, c_bigrammes, taille):
    c_info_mutuelle = {}
    for bigramme in c_bigrammes.keys():
        c_info_mutuelle[bigramme] = log(taille * c_bigrammes[bigramme] / 
                                        (c_mots[bigramme[0]] * c_mots[bigramme[1]]), 2.0)
    return c_info_mutuelle

On ordonne les bigrammes par valeur d'information mutuelle décroissante

In [32]:
c_info_mutuelle_ordonnée_par_corpus = []
c_info_mutuelle_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_info_mutuelle = info_mutuelle(c_mots, c_bigrammes, len(texte_découpé))
    c_info_mutuelle_par_corpus += [c_info_mutuelle]
    c_info_mutuelle_ordonnée = sorted(c_info_mutuelle.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_info_mutuelle_ordonnée[:10])
    c_info_mutuelle_ordonnée_par_corpus += [c_info_mutuelle_ordonnée]

===DÉCÉNNIE=== 1
   [(('sydviet', 'nams'), 18.702920061720967), (('salmonella', 'infektion'), 18.702920061720967), (('pari', 'tetslånen'), 18.702920061720967), (('osä', 'kerheten'), 18.702920061720967), (('baletten', 'romeo'), 18.702920061720967), (('inbjudna', 'föreställ'), 18.702920061720967), (('pauserna', 'inbjudes'), 18.702920061720967), (('intaga', 'förfriskningar'), 18.702920061720967), (('publika', 'utrymmen'), 18.702920061720967), (('gäst', 'teckningslistor'), 18.702920061720967)]
===DÉCÉNNIE=== 2
   [(('flo', 'rén'), 18.613965676332178), (('diumhemmets', 'cancerforskningslabora'), 18.613965676332178), (('cancerforskningslabora', 'torium'), 18.613965676332178), (('yrkesorkestrar', 'framträdanden'), 18.613965676332178), (('musikut', 'budet'), 18.613965676332178), (('ordinära', 'hushållskonsumtionen'), 18.613965676332178), (('elran', 'soneringsnämnden'), 18.613965676332178), (('förlett', 'industriminister'), 18.613965676332178), (('felbedömningen', 'höstregnen'), 18.613965676332

L'information mutuelle des mots plus fréquents avec un seuil. Le seuil est assez haut pour le pas avoir trop d'affichages

In [33]:
seuil = 200
for i, c_mots, c_bigrammes, c_info_mutuelle, c_info_mutuelle_ordonnée in zip(
    range(len(textes)),
    c_mots_par_corpus,
    c_bigrammes_par_corpus,
    c_info_mutuelle_par_corpus,
    c_info_mutuelle_ordonnée_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    for bigramme_im in c_info_mutuelle_ordonnée:
        if c_bigrammes[bigramme_im[0]] >= seuil:
            #mots_du_bigramme = bigramme_im[0].split()
            print('  ', c_info_mutuelle[bigramme_im[0]], "\t", 
                  bigramme_im[0], "\t", c_bigrammes[bigramme_im[0]], "\t",
                  c_mots[bigramme_im[0][0]], "\t", c_mots[bigramme_im[0][1]])

===DÉCÉNNIE=== 1
   10.664001072428661 	 ('bl', 'a') 	 211 	 211 	 263
   10.091895264413612 	 ('maj', 'ts') 	 221 	 391 	 221
   10.048826542521725 	 ('kungl', 'maj') 	 363 	 374 	 391
   8.33322073241565 	 ('januari', '1970') 	 417 	 530 	 1041
   8.240373855843286 	 ('februari', '1970') 	 256 	 347 	 1041
   7.788799950201567 	 ('m', 'fl') 	 677 	 1690 	 773
   6.534463638396202 	 ('herr', 'talman') 	 956 	 4486 	 981
   6.315979816396654 	 ('nr', '3') 	 216 	 2678 	 432
   6.135453450276567 	 ('bifall', 'till') 	 201 	 207 	 5894
   6.119729814888405 	 ('onsdagen', 'den') 	 455 	 456 	 6123
   6.118063940231146 	 ('torsdagen', 'den') 	 298 	 299 	 6123
   6.077895112099625 	 ('nr', '2') 	 209 	 2678 	 493
   5.978751921631565 	 ('svar', 'på') 	 259 	 372 	 4711
   5.8760715794139715 	 ('jag', 'tror') 	 235 	 4505 	 379
   5.84853155325895 	 ('chefen', 'för') 	 289 	 300 	 7134
   5.603288801844571 	 ('herr', 'statsrådet') 	 254 	 4486 	 497
   5.570042366399727 	 ('en', 'sådan') 	 

Les bigrammes avec la séquence qu'on recherche ordonnés par information mutuelle 

In [34]:
for i, corpus in enumerate(c_info_mutuelle_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('privatsko', 'lors'), 17.702920061720967), (('privatskoledödens', 'tide'), 17.702920061720967), (('fåtal', 'privatpersoner'), 16.702920061720967), (('privatsko', 'lorna'), 16.702920061720967), (('privatpersoner', 'erfarenheten'), 16.11795756099981), (('livaktig', 'privat'), 14.895565139663361), (('privat', 'arkitektverksamhet'), 14.895565139663361), (('privatpraktiserande', 'läkare'), 14.615457220470624), (('vanliga', 'privatkapitalister'), 14.454992548277378), (('privat', 'kapitalister'), 13.895565139663361), (('privata', 'företagsformer'), 13.34536805710288), (('privata', 'konsumtio'), 13.34536805710288), (('privata', 'försäkringsspa'), 13.34536805710288), (('stimulera', 'privatskole'), 13.34536805710288), (('privatskole', 'verksamheten'), 13.34536805710288), (('privat', 'företagar'), 13.310602638942203), (('regeringens', 'privatsak'), 12.636830871263193), (('privat', 'regi'), 12.573637044775998), (('privata', 'kapitalbildningen'), 12.34536805710288), (('nadsverks

## t-scores

Définition des t-scores

In [35]:
def t_scores(words, freq_unigrams, freq_bigrams):
    ts = {}
    for bigram in freq_bigrams:
        ts[bigram] = ((freq_bigrams[bigram] -
                      freq_unigrams[bigram[0]] *
                      freq_unigrams[bigram[1]] /
                      len(words)) /
                      sqrt(freq_bigrams[bigram]))
    return ts

On calcule les t-scores et on affiche les plus hauts

In [36]:
c_t_scores_ordonnée_par_corpus = []
c_t_scores_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    # print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_t_scores = t_scores(texte_découpé, c_mots, c_bigrammes)
    c_t_scores_par_corpus += [c_t_scores]
    c_t_scores_ordonnée = sorted(c_t_scores.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_t_scores_ordonnée[:10])
    c_t_scores_ordonnée_par_corpus += [c_t_scores_ordonnée]

===DÉCÉNNIE=== 1
   [(('det', 'är'), 35.291629051702536), (('herr', 'talman'), 30.585700865281023), (('av', 'herr'), 29.345576356052074), (('för', 'att'), 26.57595220196471), (('m', 'fl'), 25.901562829412054), (('att', 'det'), 25.74169969313452), (('att', 'man'), 22.020630963129907), (('onsdagen', 'den'), 21.023979793628374), (('fl', 'om'), 20.64324878438371), (('januari', '1970'), 20.357261104944765)]
===DÉCÉNNIE=== 2
   [(('det', 'är'), 34.21537624997892), (('av', 'herr'), 31.72842202104419), (('herr', 'talman'), 29.286211518054657), (('att', 'det'), 25.391966170654342), (('för', 'att'), 25.244981423215492), (('m', 'm'), 23.07571782373265), (('m', 'fl'), 22.368434889460694), (('onsdagen', 'den'), 21.667439422225677), (('kungl', 'maj'), 21.627057897016748), (('att', 'man'), 21.098338477797416)]


Les bigrammes avec la séquence qu'on recherche ordonnés par t-scores 

In [37]:
for i, corpus in enumerate(c_t_scores_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('den', 'privata'), 3.1392418586305433), (('privata', 'eller'), 2.4186977395853226), (('det', 'privata'), 2.3320196379055758), (('de', 'privata'), 1.7961611559886295), (('privata', 'och'), 1.5649868648307894), (('med', 'privata'), 1.5382274176816908), (('privat', 'företagsamhet'), 1.4137727774880409), (('privata', 'stiftelser'), 1.4133982760443484), (('privata', 'sparandet'), 1.4114280007498774), (('intressen', 'privata'), 1.4103409523115484), (('privata', 'näringslivet'), 1.4102050712567573), (('mot', 'privatskolorna'), 1.4068362153495584), (('privata', 'företag'), 1.3989269437090954), (('privat', 'eller'), 1.3960021868590187), (('ett', 'privat'), 1.3557747662967146), (('till', 'privatskolorna'), 1.3458455067807595), (('privatskolor', 'i'), 1.334275732091107), (('privatsko', 'lors'), 0.9999953130529133), (('privatskoledödens', 'tide'), 0.9999953130529133), (('fåtal', 'privatpersoner'), 0.9999906261058266), (('privatsko', 'lorna'), 0.9999906261058266), (('privatperso

## Log du rapport de vraisemblance (log-likelihood-ration)

Définition du rapport de vraisemblance

In [38]:
def likelihood_ratio(words, freq_unigrams, freq_bigrams):
    lr = {}
    for bigram in freq_bigrams:
        p = freq_unigrams[bigram[1]] / len(words)
        p1 = freq_bigrams[bigram] / freq_unigrams[bigram[0]]
        p2 = ((freq_unigrams[bigram[1]] - freq_bigrams[bigram])
              / (len(words) - freq_unigrams[bigram[0]]))
        if p1 != 1.0 and p2 != 0.0:
            lr[bigram] = 2.0 * (
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p1) +
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p2) -
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p) -
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p))
    return lr

In [39]:
def log_f(k, N, p):
    return k * log(p) + (N - k) * log(1 - p)

On le calcule

In [40]:
c_lr_ordonné_par_corpus = []
c_lr_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_lr = likelihood_ratio(texte_découpé, c_mots, c_bigrammes)
    c_lr_par_corpus += [c_lr]
    c_lr_ordonné = sorted(c_lr.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_lr_ordonné[:10])
    c_lr_ordonné_par_corpus += [c_lr_ordonné]

===DÉCÉNNIE=== 1
   [(('herr', 'talman'), 8694.871035066868), (('m', 'fl'), 7225.906531749281), (('kungl', 'maj'), 5557.287718331205), (('det', 'är'), 5479.902051361183), (('januari', '1970'), 4663.24091213827), (('onsdagen', 'den'), 3882.0824284229966), (('av', 'herr'), 3524.7125182654563), (('fl', 'om'), 2865.558020215918), (('februari', '1970'), 2749.880759729469), (('torsdagen', 'den'), 2530.631384813787)]
===DÉCÉNNIE=== 2
   [(('herr', 'talman'), 7797.323418918478), (('kungl', 'maj'), 6752.249697816897), (('m', 'fl'), 5137.393504257348), (('det', 'är'), 5137.238280022044), (('av', 'herr'), 4397.139079072011), (('onsdagen', 'den'), 4058.9614381480496), (('januari', '1970'), 3830.29967836008), (('m', 'm'), 3635.7638672764297), (('maj', 't'), 2427.5884790459513), (('anledning', 'av'), 2339.596565733067)]


Les bigrammes avec la séquence qu'on recherche ordonnés par rapports de vraisemblance

In [41]:
for i, corpus in enumerate(c_lr_ordonné_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

===DÉCÉNNIE=== 1
[(('den', 'privata'), 46.567300005693255), (('privata', 'eller'), 41.61468580585097), (('privat', 'företagsamhet'), 28.813767613814605), (('privata', 'stiftelser'), 26.288753800682343), (('privatsko', 'lorna'), 22.382577517049288), (('privata', 'sparandet'), 21.125011972708762), (('intressen', 'privata'), 19.78184401128908), (('privata', 'näringslivet'), 19.641829969483297), (('det', 'privata'), 18.51533092497084), (('privat', 'kapitalister'), 17.950300548497648), (('mot', 'privatskolorna'), 17.680092107387352), (('privat', 'företagar'), 16.903865192626817), (('nadsverksamheten', 'privata'), 15.752795899059834), (('privata', 'kapitalbildningen'), 15.752795898929719), (('privat', 'regi'), 15.719047830663996), (('privat', 'sektor'), 14.98156547422954), (('privata', 'intressenter'), 14.706487099293348), (('privata', 'företag'), 14.267187680684856), (('bedriva', 'privat'), 14.221717259757554), (('privata', 'konsumtionen'), 14.0270784402544), (('privat', 'eller'), 13.758683